# TAF4

data derived from [Janssen BDE,  et al (2022) De novo putative loss-of-function variants in TAF4 are associated with a neuro-developmental disorder. Hum Mutat;43:1844-1851. PMID:35904126](https://pubmed.ncbi.nlm.nih.gov/35904126/)

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import HTML, display
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.9.71


In [2]:
PMID = "PMID:35904126"
title = "De novo putative loss-of-function variants in TAF4 are associated with a neuro-developmental disorder"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-03-06


In [3]:
df = pd.read_excel("input/TAF4_Jansen_2022.xlsx")

In [4]:
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['individual_id'] = dft.index 
dft.head(3)

Case,Method of variant detection,Gene,Transcript,cDNA,Coding Effect,Exon,Genomic Location,Inheritance mode,Status parents,Other genetic findings / exclusions,...,Other_dysmorphology,Hand/feet,Cardiac anomalies,Kidney anomalies,Other_organ,Joint laxity,Spine,Other_skeletal,Other,individual_id
Proband 1,Trio-WES,TAF4,NM_003185.3,c.1348C>T,p.(Gln450*),1,Chr20(GRCh37):g.60639519G>A,de novo,Unaffected,NaN,...,-,Bilateral single palamar creases,Aortic dilatation,Dilated ureters,"Undescened testes, Hypermetropia","Yes, large and small joints",Normal,Tight tendon achilles,-,Proband 1
Proband 2,Trio-WGS,TAF4,NM_003185.3,c.2029C>T,p.(Gln677*),7,Chr20(GRCh37):g.60581760G>A,de novo,Unaffected,"Normal micorarray, nil else",...,Prominent midface,"Clinodactyly bilateral 5th digit of hands, but father has the same. Right foot 3-4 toe clinodactyly.","No, echo - normal",No,NaN,No. Mild flexion contractures at knees,"Lumbar hyperlordosis, mild thoracolumbar scoliosis (Cobb angle 14 degrees) at 9.5yr",External tibial torsion of right leg,NaN,Proband 2
Proband 3,Trio-WES,TAF4,NM_003185.3,c.2185C>T,p.(Gln729*),7 (out of 15),Chr20(GRCh37):g.60581604G>A,de novo,Unaffected,NaN,...,prominent midface,Clinodactylie dig 3-5 feet,absent,unknown,NaN,"Yes, in elbows",Hyperlordosis,none,"Premature adrenarche, drooling",Proband 3


In [5]:
#res = OptionColumnMapper.autoformat(df=dft, concept_recognizer=hpo_cr)
#print(res)
column_mapper_list = list()

In [6]:
abnormal_growth_d = {
 'Microcephaly (as above)': 'Microcephaly',
 'Tall and very slender': 'Tall stature',
 'long arm span': 'Increased arm span',
 'Short stature': 'Short stature'}
excluded = {}
abnormal_growthMapper = OptionColumnMapper(column_name="Abnormal growth", concept_recognizer=hpo_cr, option_d=abnormal_growth_d, excluded_d=excluded)
column_mapper_list.append(abnormal_growthMapper)
abnormal_growthMapper.preview_column(dft)

,mapping,count
0,Microcephaly (HP:0000252) (observed),1
1,Tall stature (HP:0000098) (observed),1
2,Increased arm span (HP:0012771) (observed),1
3,Short stature (HP:0004322) (observed),1


In [7]:
intellectual_disability_d = {'Yes': 'Intellectual disability',
 'mild to moderate': 'Intellectual disability',
 'According to WPPSI-IV at the age of 6y5mo intellectually slightly delayed': 'Intellectual disability, mild',
 #'cognitive profile uneven': 'PLACEHOLDER',
 'severe ; smiling 9 months': 'Global developmental delay',
 'Moderate ID': 'Intellectual disability, moderate',
 'yes': 'Intellectual disability'}
excluded = {'No': 'Intellectual disability'}
intellectual_disabilityMapper = OptionColumnMapper(column_name="Intellectual disability", 
                                                   concept_recognizer=hpo_cr, option_d=intellectual_disability_d, excluded_d=excluded)
column_mapper_list.append(intellectual_disabilityMapper)
intellectual_disabilityMapper.preview_column(dft)

,mapping,count
0,Intellectual disability (HP:0001249) (observed),6
1,Intellectual disability (HP:0001249) (excluded),1
2,"Intellectual disability, mild (HP:0001256) (observed)",1
3,"Intellectual disability, moderate (HP:0002342) (observed)",1


In [8]:
autism_d = {'Yes': 'Autism',
 'mild to moderate': 'Autism',
 'Autistic features': 'Autistic behavior',
 'but not enough for diagnosis': 'Autistic behavior',
 'yes autistic features': 'Autistic behavior'}
excluded = {'No': 'Autism',}
autismMapper = OptionColumnMapper(column_name="Autism", concept_recognizer=hpo_cr, option_d=autism_d, excluded_d=excluded)
column_mapper_list.append(autismMapper)
autismMapper.preview_column(dft)

,mapping,count
0,Autism (HP:0000717) (observed),2
1,Autism (HP:0000717) (excluded),2
2,Autistic behavior (HP:0000729) (observed),3


In [9]:
other_behaviour_psychiatric_abnormality_d = {'Very shy': 'Shyness',
 'Can be agressive': 'Aggressive behavior',
 'PICA': 'Pica',
 'automutilation': 'Self-mutilation',
 'sleeping disorder and self hugging': 'Self hugging',
 'Shy': 'Shyness',
}
excluded = {}
other_behaviour_psychiatric_abnormalityMapper = OptionColumnMapper(column_name="Other Behaviour/Psychiatric abnormality", concept_recognizer=hpo_cr, option_d=other_behaviour_psychiatric_abnormality_d, excluded_d=excluded)
column_mapper_list.append(other_behaviour_psychiatric_abnormalityMapper)
other_behaviour_psychiatric_abnormalityMapper.preview_column(dft)

,mapping,count
0,Shyness (HP:0100962) (observed),3
1,Aggressive behavior (HP:0000718) (observed),1
2,Pica (HP:0011856) (observed),1
3,Self-mutilation (HP:0000742) (observed),1
4,Self hugging (HP:0032521) (observed),1


In [10]:
sitting_unaided__d = {
 '10 mo': 'Delayed ability to sit',
 '1 yr': 'Delayed ability to sit',
 '18 months': 'Delayed ability to sit',
 '15 months': 'Delayed ability to sit'}
excluded = { '7 months': 'Delayed ability to sit',}
sitting_unaided_Mapper = OptionColumnMapper(column_name="Sitting unaided", concept_recognizer=hpo_cr, option_d=sitting_unaided__d, excluded_d=excluded)
column_mapper_list.append(sitting_unaided_Mapper)
sitting_unaided_Mapper.preview_column(dft)

,mapping,count
0,Delayed ability to sit (HP:0025336) (observed),4
1,Delayed ability to sit (HP:0025336) (excluded),1


In [11]:
walking_unaided_d = {
 '2 yr 2 months': 'Delayed ability to walk',
 '3 years': 'Delayed ability to walk',
 '2 years': 'Delayed ability to walk',}
excluded = { }
walking_unaidedMapper = OptionColumnMapper(column_name="Walking unaided", concept_recognizer=hpo_cr, option_d=walking_unaided_d, excluded_d=excluded)
column_mapper_list.append(walking_unaidedMapper)
walking_unaidedMapper.preview_column(dft)

,mapping,count
0,Delayed ability to walk (HP:0031936) (observed),3


In [12]:
language_development__d = {
 'Delayed initially. Only 4 words at 2.5 yr': 'Delayed speech and language development',
 'delayed': 'Delayed speech and language development',
 'At the age of 3 years ~50 words and a few two-word sentences. At the age of 5 years around 100 words and does not understand more complex sentences': 'Delayed speech and language development',
 'Expressive and receptive speech delay': 'Delayed speech and language development',
 '2-3 word sentences at age 12yr': 'Delayed speech and language development',
 'Moderate receptive speech delay': 'Delayed speech and language development',
 'severe expressive delay': 'Expressive language delay',
 'severe ongoing dyspraxia. Required long-term speech therapy. Speaks in sentences but can be hard to understand.': 'Delayed speech and language development'}
excluded = { 'no verbral apraxia': 'Speech apraxia',}
language_development_Mapper = OptionColumnMapper(column_name="Language development", concept_recognizer=hpo_cr, option_d=language_development__d, excluded_d=excluded)
column_mapper_list.append(language_development_Mapper)
language_development_Mapper.preview_column(dft)

,mapping,count
0,Delayed speech and language development (HP:0000750) (observed),8
1,Apraxia (HP:0002186) (observed),1
2,Expressive language delay (HP:0002474) (observed),1


In [13]:
seizures___epilepsy_d = {
 'Yes - from 8 months. Febrile seizures x5 and one non-febrile seizure': 'Febrile seizure (within the age range of 3 months to 6 years)',
 'febrile seizure 1x': 'Febrile seizure (within the age range of 3 months to 6 years)',
 }
excluded = {'no': 'Seizure', 'No': 'Seizure',}
seizures___epilepsyMapper = OptionColumnMapper(column_name="Seizures / Epilepsy", concept_recognizer=hpo_cr, option_d=seizures___epilepsy_d, excluded_d=excluded)
column_mapper_list.append(seizures___epilepsyMapper)
seizures___epilepsyMapper.preview_column(dft)

,mapping,count
0,Seizure (HP:0001250) (excluded),3
1,Febrile seizure (within the age range of 3 months to 6 years) (HP:0002373) (observed),2


In [14]:
mri_ct_d = {
 'MRI X2 (age 5y and 6y9m) - mild superior cerebellar vermis atrophy': 'Cerebellar vermis atrophy',
 'and inferior vermian hypoplasia. Stable over 2 scans. (Images provided in supplemental clinical histories)': 'Cerebellar vermis hypoplasia',}
excluded = {}
mri_ctMapper = OptionColumnMapper(column_name="MRI/CT", concept_recognizer=hpo_cr, option_d=mri_ct_d, excluded_d=excluded)
column_mapper_list.append(mri_ctMapper)
mri_ctMapper.preview_column(dft)

,mapping,count
0,Cerebellar vermis atrophy (HP:0006855) (observed),1
1,Cerebellar vermis hypoplasia (HP:0001320) (observed),1


In [15]:
other_neurological_findings_d = {
 'Walks on tiptoes. Spasticity in legs': 'Spasticity',
 'toe walking': 'Tip-toe gait',
 'handflapping': 'Recurrent hand flapping',
 'with increased extensor tone as a baby. Later hypotonic.': 'Hypotonia',
}
excluded = {}
other_neurological_findingsMapper = OptionColumnMapper(column_name="Other neurological findings", concept_recognizer=hpo_cr, option_d=other_neurological_findings_d, excluded_d=excluded)
column_mapper_list.append(other_neurological_findingsMapper)
other_neurological_findingsMapper.preview_column(dft)

,mapping,count
0,Spasticity (HP:0001257) (observed),1
1,Tip-toe gait (HP:0030051) (observed),1
2,Recurrent hand flapping (HP:0100023) (observed),1
3,Hypotonia (HP:0001252) (observed),1


In [16]:
nose_d = {'Prominent nasal bridge': 'Prominent nasal bridge',
 'convex nose': 'Convex nasal ridge',
 'Prominent nose': 'Prominent nose',
 'high nasal bridge. Not beaked. Alae - normal': 'Prominent nasal bridge',
 'Long nose': 'Long nose',
 'high nasal bridge': 'Prominent nasal bridge',
 'slightly beaked': 'Convex nasal ridge',
 'underdeveloped alae nasi': 'Underdeveloped nasal alae',
 'Small upturned nose with wide nose tip; rather wide and prominent nasal bridge': 'Wide nose',
 'Low nasal bridge. Short columella.': 'Short columella',
 'low and wide nasal bridge': 'Wide nasal bridge',
 'upturned nasal tip': 'Anteverted nares',
 'convex profile': 'Convex nasal ridge',
 'high nasal bridge: had surgery for deviated nasal septum in teens': 'Prominent nasal bridge',
}
excluded = {}
noseMapper = OptionColumnMapper(column_name="Nose", concept_recognizer=hpo_cr, option_d=nose_d, excluded_d=excluded)
column_mapper_list.append(noseMapper)
noseMapper.preview_column(dft)

,mapping,count
0,Prominent nasal bridge (HP:0000426) (observed),4
1,Convex nasal ridge (HP:0000444) (observed),3
2,Prominent nose (HP:0000448) (observed),1
3,Long nose (HP:0003189) (observed),2
4,Underdeveloped nasal alae (HP:0000430) (observed),1
5,Wide nose (HP:0000445) (observed),1
6,Short columella (HP:0002000) (observed),1
7,Wide nasal bridge (HP:0000431) (observed),1
8,Anteverted nares (HP:0000463) (observed),1


In [17]:
mouth_palate_d = {'Tounge tie': 'Ankyloglossia',
 'Full lips': 'Thick vermilion border',
 'overbite': 'Increased overbite',
 'sligntly full lower vermillion': 'Thick lower lip vermilion',
 'Higher palate': 'High palate',
 'spacing teeth': 'Widely spaced teeth',
 'protruding lower lip': 'Everted lower lip vermilion',
 'carpe shaped mouth': 'U-Shaped upper lip vermilion',
 'Cleft soft palate + 1/3 hard palate; repaired age 7 months; mild oligodontia upper and lower dentition; Thin upper lip': 'Cleft soft palate',}
excluded = {}
mouth_palateMapper = OptionColumnMapper(column_name="Mouth/Palate", concept_recognizer=hpo_cr, option_d=mouth_palate_d, excluded_d=excluded)
column_mapper_list.append(mouth_palateMapper)
mouth_palateMapper.preview_column(dft)

,mapping,count
0,Ankyloglossia (HP:0010296) (observed),1
1,Thick vermilion border (HP:0012471) (observed),1
2,Increased overbite (HP:0011094) (observed),1
3,Thick lower lip vermilion (HP:0000179) (observed),1
4,High palate (HP:0000218) (observed),1
5,Widely spaced teeth (HP:0000687) (observed),1
6,Everted lower lip vermilion (HP:0000232) (observed),1
7,U-Shaped upper lip vermilion (HP:0010806) (observed),1
8,Cleft soft palate (HP:0000185) (observed),1


In [18]:
philtrum_d = {'Prominent': 'Deep philtrum',
 'Short': 'Short philtrum',
 'Short philtrum': 'Short philtrum',
 'Rather long and deep philtrum': 'Deep philtrum',
 'Flat': 'Smooth philtrum',
 'prominent philtrum': 'Deep philtrum',
 'wide philtrum': 'Broad philtrum',
}
excluded = {}
philtrumMapper = OptionColumnMapper(column_name="Philtrum", concept_recognizer=hpo_cr, option_d=philtrum_d, excluded_d=excluded)
column_mapper_list.append(philtrumMapper)
philtrumMapper.preview_column(dft)

,mapping,count
0,Deep philtrum (HP:0002002) (observed),3
1,Short philtrum (HP:0000322) (observed),2
2,Smooth philtrum (HP:0000319) (observed),1
3,Broad philtrum (HP:0000289) (observed),1


In [19]:
eyes_palpebral_fissures_d = {
 'long palpebral fissures': 'Long palpebral fissure',
 'Deep set eyes': 'Deeply set eye',
 'a little widely spaced and upslanted': 'Upslanted palpebral fissure',
 'bilateral epiblepharon': 'Epiblepharon',
 'Short': 'Short palpebral fissure',
 'slightly upslanting palpebral fissures. Synophrys.': 'Upslanted palpebral fissure',
 'Narrow palpebral fissures vertically (not  short)': 'Narrow palpebral fissure',
 'Downward slanting palpebral fissures': 'Downslanted palpebral fissures'}
excluded = {}
eyes_palpebral_fissuresMapper = OptionColumnMapper(column_name="Eyes/palpebral fissures", concept_recognizer=hpo_cr, option_d=eyes_palpebral_fissures_d, excluded_d=excluded)
column_mapper_list.append(eyes_palpebral_fissuresMapper)
eyes_palpebral_fissuresMapper.preview_column(dft)

,mapping,count
0,Long palpebral fissure (HP:0000637) (observed),1
1,Deeply set eye (HP:0000490) (observed),1
2,Upslanted palpebral fissure (HP:0000582) (observed),2
3,Epiblepharon (HP:0011225) (observed),1
4,Short palpebral fissure (HP:0012745) (observed),2
5,Narrow palpebral fissure (HP:0045025) (observed),1
6,Downslanted palpebral fissures (HP:0000494) (observed),1


In [20]:
ears_d = {'Large': 'Macrotia',
 'anteverted ears': 'Anteverted ears',
 'bigger than normal': 'Macrotia',
 'posteriorly rotated': 'Posteriorly rotated ears',
 'Small': 'Microtia',
 'low set': 'Low-set ears',
 'posteriorly rotated ears': 'Posteriorly rotated ears'}
excluded = {}
earsMapper = OptionColumnMapper(column_name="Ears", concept_recognizer=hpo_cr, option_d=ears_d, excluded_d=excluded)
column_mapper_list.append(earsMapper)
earsMapper.preview_column(dft)

,mapping,count
0,Macrotia (HP:0000400) (observed),2
1,Anteverted ears (HP:0040080) (observed),1
2,Posteriorly rotated ears (HP:0000358) (observed),2
3,Recurrent otitis media (HP:0000403) (observed),1
4,Conductive hearing impairment (HP:0000405) (observed),1
5,Microtia (HP:0008551) (observed),1
6,Low-set ears (HP:0000369) (observed),1


In [21]:
eyebrows_forehead_d = {
 'Synophrys': 'Synophrys',
 'mildly arched': 'Highly arched eyebrow',
 'Arched eyebrows': 'Highly arched eyebrow',
 'Rather thick and a little arched eyebrows': 'Highly arched eyebrow',
 'thick eyebrows': 'Thick eyebrow',
 'Mildly arched eyebrows': 'Highly arched eyebrow',
 }
excluded = {}
eyebrows_foreheadMapper = OptionColumnMapper(column_name="Eyebrows/forehead", concept_recognizer=hpo_cr, option_d=eyebrows_forehead_d, excluded_d=excluded)
column_mapper_list.append(eyebrows_foreheadMapper)
eyebrows_foreheadMapper.preview_column(dft)

,mapping,count
0,Synophrys (HP:0000664) (observed),1
1,Highly arched eyebrow (HP:0002553) (observed),5
2,Thick eyebrow (HP:0000574) (observed),1


In [22]:
chin_jaw__d = {'Small chin': 'Short chin',
 'Relative retrognathia': 'Retrognathia',
 'Mild retrognathia': 'Retrognathia',
}
excluded = {}
chin_jaw_Mapper = OptionColumnMapper(column_name="Chin/Jaw", concept_recognizer=hpo_cr, option_d=chin_jaw__d, excluded_d=excluded)
column_mapper_list.append(chin_jaw_Mapper)
chin_jaw_Mapper.preview_column(dft)

,mapping,count
0,Short chin (HP:0000331) (observed),1
1,Retrognathia (HP:0000278) (observed),2


In [23]:
other_d = {
 'Inguinal hernia repaired age 3 months': 'Inguinal hernia',}
excluded = {}
otherMapper = OptionColumnMapper(column_name="Other_dysmorphology", concept_recognizer=hpo_cr, option_d=other_d, excluded_d=excluded)
column_mapper_list.append(otherMapper)
otherMapper.preview_column(dft)

,mapping,count
0,Inguinal hernia (HP:0000023) (observed),1


In [24]:
hand_feet_d = {'Bilateral single palamar creases': 'Bilateral single transverse palmar creases',
 'Clinodactyly bilateral 5th digit of hands': 'Clinodactyly of the 5th finger',
 'but father has the same. Right foot 3-4 toe clinodactyly.': 'Toe clinodactyly',
 'Clinodactylie dig 3-5 feet': 'Toe clinodactyly',
 'Flat feet': 'Pes planus',
 'Prominent interphalangeal joints': 'Prominent interphalangeal joints',
 'mild flexion contractures fingers': 'Flexion contracture',
 'moderate 2-3 syndactyly toes: mother has unilateral 2-3 syndactyly': '2-3 toe syndactyly',}
excluded = {}
hand_feetMapper = OptionColumnMapper(column_name="Hand/feet", concept_recognizer=hpo_cr, option_d=hand_feet_d, excluded_d=excluded)
column_mapper_list.append(hand_feetMapper)
hand_feetMapper.preview_column(dft)

,mapping,count
0,Bilateral single transverse palmar creases (HP:0007598) (observed),1
1,Clinodactyly of the 5th finger (HP:0004209) (observed),1
2,Toe clinodactyly (HP:0001863) (observed),2
3,Pes planus (HP:0001763) (observed),1
4,Prominent interphalangeal joints (HP:0006237) (observed),1
5,Flexion contracture (HP:0001371) (observed),1
6,2-3 toe syndactyly (HP:0004691) (observed),1


In [25]:
cardiac_anomalies_d = {'Aortic dilatation': 'Aortic aneurysm',
 }
excluded = {}
cardiac_anomaliesMapper = OptionColumnMapper(column_name="Cardiac anomalies", concept_recognizer=hpo_cr, option_d=cardiac_anomalies_d, excluded_d=excluded)
column_mapper_list.append(cardiac_anomaliesMapper)
cardiac_anomaliesMapper.preview_column(dft)

,mapping,count
0,Aortic aneurysm (HP:0004942) (observed),1


In [26]:
kidney_anomalies_d = {'Dilated ureters': 'Hydroureter',}
excluded = {}
kidney_anomaliesMapper = OptionColumnMapper(column_name="Kidney anomalies", concept_recognizer=hpo_cr, option_d=kidney_anomalies_d, excluded_d=excluded)
column_mapper_list.append(kidney_anomaliesMapper)
kidney_anomaliesMapper.preview_column(dft)

,mapping,count
0,Hydroureter (HP:0000072) (observed),1


In [27]:
other_d = {'Undescened testes': 'Cryptorchidism',
 'Hypermetropia': 'Hypermetropia',
 'Hypermetrophy': 'Hypermetropia',
 'Severe constipation': 'Constipation',}
excluded = {}
otherOrganMapper = OptionColumnMapper(column_name="Other_organ", concept_recognizer=hpo_cr, option_d=other_d, excluded_d=excluded)
column_mapper_list.append(otherOrganMapper)
otherOrganMapper.preview_column(dft)

,mapping,count
0,Cryptorchidism (HP:0000028) (observed),1
1,Hypermetropia (HP:0000540) (observed),2
2,Constipation (HP:0002019) (observed),1


In [28]:
joint_laxity__d = {'Yes': 'Joint hypermobility',
 'large and small joints': 'Joint hypermobility',
 'No. Mild flexion contractures at knees': 'Knee flexion contracture',

 'Overall mild joint hypermobility': 'Joint hypermobility',
 'flat feet': 'Pes planus',
}
excluded = {}
joint_laxity_Mapper = OptionColumnMapper(column_name="Joint laxity", concept_recognizer=hpo_cr, option_d=joint_laxity__d, excluded_d=excluded)
column_mapper_list.append(joint_laxity_Mapper)
joint_laxity_Mapper.preview_column(dft)

,mapping,count
0,Joint hypermobility (HP:0001382) (observed),4
1,Knee flexion contracture (HP:0006380) (observed),1
2,Pes planus (HP:0001763) (observed),1


In [29]:
spine_d = {
 'Lumbar hyperlordosis': 'Lumbar hyperlordosis',
 'mild thoracolumbar scoliosis (Cobb angle 14 degrees) at 9.5yr': 'Thoracolumbar scoliosis',
 'Hyperlordosis': 'Hyperlordosis',
 'Kyphosis': 'Kyphosis',
 'mild scoliosis': 'Scoliosis',
 'narrow thorax: no vertebral anomalies on Xray': 'Narrow chest',
}
excluded = {}
spineMapper = OptionColumnMapper(column_name="Spine", concept_recognizer=hpo_cr, option_d=spine_d, excluded_d=excluded)
column_mapper_list.append(spineMapper)
spineMapper.preview_column(dft)

,mapping,count
0,Lumbar hyperlordosis (HP:0002938) (observed),1
1,Thoracolumbar scoliosis (HP:0002944) (observed),1
2,Hyperlordosis (HP:0003307) (observed),1
3,Kyphosis (HP:0002808) (observed),1
4,Scoliosis (HP:0002650) (observed),1
5,Narrow chest (HP:0000774) (observed),1


In [30]:
other_d = {
 'External tibial torsion of right leg': 'External tibial torsion',
 'limited elbow extension': 'Limited elbow extension',
 'anterior axillary webbing': 'Axillary pterygium',
 'Delayed bone age': 'Delayed skeletal maturation'}
excluded = {}
otherMapper = OptionColumnMapper(column_name="Other_skeletal", concept_recognizer=hpo_cr, option_d=other_d, excluded_d=excluded)
column_mapper_list.append(otherMapper)
otherMapper.preview_column(dft)

,mapping,count
0,External tibial torsion (HP:0034373) (observed),1
1,Limited elbow extension (HP:0001377) (observed),1
2,Axillary pterygium (HP:0001060) (observed),1
3,Delayed skeletal maturation (HP:0002750) (observed),1


In [31]:
other_d = {
 'Premature adrenarche': 'Premature adrenarche',
 'drooling': 'Drooling',
 '3 cafe au lait spots': 'Cafe-au-lait spot',
 'Susceptible to colds. Feeding difficulties in infancy. Gagged on liquids and foods.': 'Feeding difficulties in infancy',
 'Feeding diiculties and drooling in infancy. Severe constipation until over 15 months': 'Drooling',
 'then resolved. Multiple pigmented naevi.': 'Melanocytic nevus',
 'Wide intermammillary distance': 'Wide intermamillary distance'}
excluded = {}
otherMapper = OptionColumnMapper(column_name="Other", concept_recognizer=hpo_cr, option_d=other_d, excluded_d=excluded)
column_mapper_list.append(otherMapper)
otherMapper.preview_column(dft)

,mapping,count
0,Premature adrenarche (HP:0012412) (observed),1
1,Drooling (HP:0002307) (observed),2
2,Cafe-au-lait spot (HP:0000957) (observed),1
3,Feeding difficulties in infancy (HP:0008872) (observed),1
4,Melanocytic nevus (HP:0000995) (observed),1
5,Wide intermamillary distance (HP:0006610) (observed),1


In [32]:
sexMapper = SexColumnMapper(column_name="Sex",male_symbol="Male", female_symbol="Female")
# sexMapper.preview_column(dft)
def to_iso8601(age):
    if age.endswith("yr"):
        age = age.replace("yr","").strip()
        return f"P{age}Y"
    elif age == "9 yr 9 mo":
        return "P9Y9M"
    else:
        raise ValueError(f"Could not map age {age}")
 
dft["iso_age"] = dft['Age at last examination'].apply(lambda x:to_iso8601(x))
ageMapper = AgeColumnMapper.iso8601(column_name="iso_age")

In [33]:
taf4_transcript = "NM_003185.4"
vmanager = VariantManager(df=dft, individual_column_name="individual_id",gene_symbol="TAF4",transcript=taf4_transcript,
                         allele_1_column_name="cDNA")

In [34]:
varMapper = VariantColumnMapper(variant_column_name="cDNA", variant_d=vmanager.get_variant_d(), default_genotype="heterozygous")

In [35]:
MRD73 = Disease(disease_id='OMIM:620450', disease_label='Intellectual developmental disorder, autosomal dominant 73')
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="individual_id", 
                        age_at_last_encounter_mapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
encoder.set_disease(MRD73)

In [36]:
individuals = encoder.get_individuals()

In [37]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,1


In [38]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Proband 1 (MALE; P14Y),"Intellectual developmental disorder, autosomal dominant 73 (OMIM:620450)",NM_003185.4:c.1348C>T (heterozygous),Joint hypermobility (HP:0001382); Hydroureter (HP:0000072); Aortic aneurysm (HP:0004942); Hypermetropia (HP:0000540); Cryptorchidism (HP:0000028); Bilateral single transverse palmar creases (HP:0007598); Short chin (HP:0000331); Deep philtrum (HP:0002002); Shyness (HP:0100962); Ankyloglossia (HP:0010296); Macrotia (HP:0000400); Long palpebral fissure (HP:0000637); Prominent nasal bridge (HP:0000426); Convex nasal ridge (HP:0000444); Intellectual disability (HP:0001249); Autism (HP:0000717); Anteverted ears (HP:0040080)
Proband 2 (FEMALE; P9Y9M),"Intellectual developmental disorder, autosomal dominant 73 (OMIM:620450)",NM_003185.4:c.2029C>T (heterozygous),Microcephaly (HP:0000252); Delayed ability to sit (HP:0025336); Delayed speech and language development (HP:0000750); Cerebellar vermis atrophy (HP:0006855); Cerebellar vermis hypoplasia (HP:0001320); Spasticity (HP:0001257); Prominent nose (HP:0000448); Prominent nasal bridge (HP:0000426); Thick vermilion border (HP:0012471); Short philtrum (HP:0000322); Synophrys (HP:0000664); Highly arched eyebrow (HP:0002553); Clinodactyly of the 5th finger (HP:0004209); Toe clinodactyly (HP:0001863); Knee flexion contracture (HP:0006380); Lumbar hyperlordosis (HP:0002938); Thoracolumbar scoliosis (HP:0002944); External tibial torsion (HP:0034373); excluded: Intellectual disability (HP:0001249); excluded: Autism (HP:0000717); excluded: Seizure (HP:0001250)
Proband 3 (FEMALE; P8Y),"Intellectual developmental disorder, autosomal dominant 73 (OMIM:620450)",NM_003185.4:c.2185C>T (heterozygous),Intellectual disability (HP:0001249); Shyness (HP:0100962); Delayed ability to sit (HP:0025336); Delayed ability to walk (HP:0031936); Delayed speech and language development (HP:0000750); Apraxia (HP:0002186); Long nose (HP:0003189); Prominent nasal bridge (HP:0000426); Convex nasal ridge (HP:0000444); Underdeveloped nasal alae (HP:0000430); Increased overbite (HP:0011094); Thick lower lip vermilion (HP:0000179); Short philtrum (HP:0000322); Highly arched eyebrow (HP:0002553); Retrognathia (HP:0000278); Toe clinodactyly (HP:0001863); Joint hypermobility (HP:0001382); Hyperlordosis (HP:0003307); Premature adrenarche (HP:0012412); Drooling (HP:0002307)
Proband 4 (FEMALE; P3Y),"Intellectual developmental disorder, autosomal dominant 73 (OMIM:620450)",NM_003185.4:c.2453dup (heterozygous),"Delayed speech and language development (HP:0000750); Epiblepharon (HP:0011225); Joint hypermobility (HP:0001382); Hypermetropia (HP:0000540); Posteriorly rotated ears (HP:0000358); Intellectual disability, mild (HP:0001256); Deeply set eye (HP:0000490); Deep philtrum (HP:0002002); Upslanted palpebral fissure (HP:0000582); Macrotia (HP:0000400); Aggressive behavior (HP:0000718); Wide nose (HP:0000445); Autistic behavior (HP:0000729); High palate (HP:0000218); Cafe-au-lait spot (HP:0000957); Highly arched eyebrow (HP:0002553); excluded: Delayed ability to sit (HP:0025336)"
Proband 5 (FEMALE; P2Y),"Intellectual developmental disorder, autosomal dominant 73 (OMIM:620450)",NM_003185.4:c.2570_2571del (heterozygous),Intellectual disability (HP:0001249); Delayed speech and language development (HP:0000750); Febrile seizure (within the age range of 3 months to 6 years) (HP:0002373); Short columella (HP:0002000); Smooth philtrum (HP:0000319); Short palpebral fissure (HP:0012745); Upslanted palpebral fissure (HP:0000582); Highly arched eyebrow (HP:0002553); Constipation (HP:0002019); Feeding difficulties in infancy (HP:0008872)
Proband 6 (FEMALE; P16Y),"Intellectual developmental disorder, autosomal dominant 73 (OMIM:620450)",NM_003185.4:c.2664del (heterozygous),Widely spaced teeth (HP:0000687); Anteverted nares (HP:0000463); Delayed ability to sit (HP:0025336); Everted lower lip vermilion (HP:0000232); Intellectual disability (HP:0001249); Delayed abil

In [39]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 8 GA4GH phenopackets to the directory phenopackets


In [40]:
created_by = metadata.get_created_by()
PMID = metadata.get_pmid()
builder = HpoaTableBuilder.from_individuals(individuals, created_by)
hpoa_creator = builder.autosomal_dominant(PMID).build()
df = hpoa_creator.get_dataframe()
df.head(2)

We found a total of 99 unique HPO terms
Extracted disease: Intellectual developmental disorder, autosomal dominant 73 (OMIM:620450)


,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620450,"Intellectual developmental disorder, autosomal dominant 73",HP:0001382,Joint hypermobility,,,3/3,,,,,PMID:35904126,PCS,ORCID:0000-0002-5648-2155
1,OMIM:620450,"Intellectual developmental disorder, autosomal dominant 73",HP:0000072,Hydroureter,,,1/1,,,,,PMID:35904126,PCS,ORCID:0000-0002-5648-2155


In [41]:
#hpoa_creator.write_data_frame()